In [ ]:
import tensorflow as tf
import math
import time
import numpy as np

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Arcface/backbone_0.py .
!cp /content/drive/MyDrive/Arcface/hypar.py .
!cp /content/drive/MyDrive/Arcface/network_16.py .
import hypar
import backbone_0 as nn
import network_16 as net

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
X = np.load('/content/drive/MyDrive/Arcface/x_train.npy', allow_pickle=True)
Y = np.load('/content/drive/MyDrive/Arcface/y_train.npy', allow_pickle=True)

In [ ]:
X = np.array(X, dtype='float32')
Y = np.array(Y, dtype='int32')
Y = np.reshape(Y, Y.shape[0])
print("X shape:",X.shape,"Y shape:",Y.shape) 
X = net.Resnet_preprocess(X)
images = X
labels = Y

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(hypar.batch_size)
print("Training dataset ready!")

X shape: (1476, 112, 112, 3) Y shape: (1476,)
Training dataset ready!


In [ ]:
num_classes = hypar.num_classes #number of people in the dataset

#feature vector dimension = (512) [comes from the resnet model]

class Arcface_Layer(tf.keras.layers.Layer):                                     # Arcface layer definition
    def __init__(self, num_outputs = num_classes, s=64., m=0.5):                 # s is scale factor, m is the margin to be added to the angle 'theta'
        super(Arcface_Layer, self).__init__()
        self.num_outputs = num_outputs
        self.s = s
        self.m = m

    def build(self, input_shape):

        self.kernel = self.add_weight(name='weight',
                                          shape=(input_shape[-1],self.num_outputs),
                                          initializer='glorot_uniform',
                                          regularizer=tf.keras.regularizers.l2(l=5e-4),
                                          trainable=True)
        super(Arcface_Layer, self).build(input_shape)


    def call(self, feature_vec, ground_truth_vec):                              # inputs is the 512 dimension feature vector
        mask = tf.one_hot(ground_truth_vec, num_classes)                        # dims of mask is (num_classes), it is a one-hot vector
        inv_mask = tf.subtract(1., mask)
        
        # feature vector and weights norm
        x = feature_vec
        norm_x = tf.norm(feature_vec, axis=1, keepdims=True)
        norm_W = tf.norm(self.kernel, axis=0, keepdims=True)
        
        x = tf.math.divide(x, norm_x)
        W = tf.math.divide(self.kernel,norm_W)
        #cos_theta = tf.matmul(tf.transpose(W),tf.transpose(x))  # logit of  W.t*x
        cos_theta = tf.matmul(x, W)
        theta = tf.math.acos(cos_theta)                                         # all angle between each class' weight and x
        theta_class = tf.multiply(theta,mask)                                   # increasing angle theta of the class x belongs to alone
        theta_class_added_margin = theta_class + self.m
        theta_class_added_margin = theta_class_added_margin*mask
        cos_theta_margin = tf.math.cos(theta_class_added_margin)
        s_cos_t = tf.multiply(self.s, cos_theta_margin)
        s_cos_j = tf.multiply(self.s,tf.multiply(cos_theta,inv_mask))

        output = tf.add(s_cos_t,s_cos_j)

        return output

In [ ]:
class train_model(tf.keras.Model):
    def __init__(self):
        super(train_model, self).__init__()
        #self.resnet = net.Resnet_nn()
        self.resnet = net.Resnet()
        self.arcface = Arcface_Layer()

    def call(self, x, y):
        x = self.resnet(x)
        return self.arcface(x, y)

# Instantiate a loss function.
def loss_fxn(logits,labels):
    loss_fn = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss_fn
    
# Instantiate an optimizer to train the model.
learning_rate = 0.0005
#optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=False)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model = train_model()

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images,labels)
        pred = tf.nn.softmax(logits)
        #inf_loss = loss_fxn(pred,labels)
        inf_loss = loss_fxn(logits,labels)
        loss = inf_loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss = tf.reduce_mean(loss)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, axis=1, output_type=tf.dtypes.int32), tf.cast(labels,dtype = tf.int32)), dtype=tf.float32))
    inference_loss = tf.reduce_mean(inf_loss)
    regularization_loss = 0
    return accuracy, train_loss, inference_loss, regularization_loss

In [ ]:
epochs = 20
reg_coef = 1.0

file_name = '/content/drive/MyDrive/Arcface/checkpoint/try4_16/model_weights_'
file_number = 10
file_name = file_name + str(file_number*epochs)+ '_epochs'
model = tf.keras.models.load_model(file_name)

In [ ]:
print(file_name)

/content/drive/MyDrive/Arcface/checkpoint/try4_16/model_weights_200_epochs


In [ ]:
random_out = model(X[0:3,:,:,:],Y[0:3])
loss_log = []

epochs = 50
reg_coef = 1.0
file_number = 5

for save_wt in range (0, 5):
  for epoch in range(epochs):
      print("\nStart of epoch %d" % (epoch,))
      start_time = time.time()

      # Iterate over the batches of the dataset.
      for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
          accuracy, train_loss, inference_loss, regularization_loss = train_step(x_batch_train, y_batch_train)
          if step % 20 == 0:
            loss_log.append(train_loss)
            print("Training loss (for one batch) at step %d: %.4f"% (step, float(train_loss)))
      
  file_number += 1
  file_name = '/content/drive/MyDrive/Arcface/checkpoint/try4_16/model_weights_'
  file_name = file_name + str(file_number*epochs)+ '_epochs'
  model.save(file_name)


Start of epoch 0
Training loss (for one batch) at step 0: 36.7603
Training loss (for one batch) at step 20: 31.7190
Training loss (for one batch) at step 40: 35.1816
Training loss (for one batch) at step 60: 34.8949
Training loss (for one batch) at step 80: 34.2491

Start of epoch 1
Training loss (for one batch) at step 0: 35.5965
Training loss (for one batch) at step 20: 35.9811
Training loss (for one batch) at step 40: 34.2415
Training loss (for one batch) at step 60: 35.7493
Training loss (for one batch) at step 80: 33.2042

Start of epoch 2
Training loss (for one batch) at step 0: 33.7470
Training loss (for one batch) at step 20: 34.9552
Training loss (for one batch) at step 40: 33.3988
Training loss (for one batch) at step 60: nan
Training loss (for one batch) at step 80: nan

Start of epoch 3
Training loss (for one batch) at step 0: nan
Training loss (for one batch) at step 20: nan
Training loss (for one batch) at step 40: nan
Training loss (for one batch) at step 60: nan
Traini

KeyboardInterrupt: ignored

In [ ]:
loss_log_data = np.array(loss_log)
np.save('/content/drive/MyDrive/Arcface/checkpoint/try4_16/loss_log/loss_log_1.npy', loss_log_data)

In [ ]:
test_model = model.resnet
result = test_model.predict(X)
result.shape

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f3305652ef0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


(1476, 512)

In [ ]:
index = []
for i in range (0,1470):
    if Y[i] == 25:
        index.append(i)

In [ ]:
index

[0,
 6,
 26,
 44,
 162,
 163,
 165,
 251,
 253,
 364,
 398,
 399,
 461,
 494,
 507,
 518,
 524,
 583,
 730,
 789,
 863,
 877,
 991,
 1002,
 1041,
 1051,
 1068,
 1158,
 1178,
 1184,
 1308,
 1415]

In [ ]:
a = np.dot(result[877,:], np.array(result[1308,:]).T)
print(a)

1440517.4


In [ ]:
from sklearn.preprocessing import normalize
results = normalize(result, axis = 0)
results.shape

In [ ]:
model.arcface.get_weights()

[array([[-0.11767528,  0.14456384, -0.02135386, ..., -0.03348209,
         -0.00698351, -0.05040066],
        [-0.03981043,  0.02772743, -0.06435528, ...,  0.09751327,
          0.06037353,  0.1113415 ],
        [ 0.04373817, -0.03364272, -0.10783768, ...,  0.06325577,
          0.03565495,  0.04472327],
        ...,
        [-0.01150434, -0.03776852, -0.04783383, ...,  0.00264406,
         -0.0351619 ,  0.05422439],
        [-0.10210086, -0.06197488, -0.05334978, ...,  0.0618397 ,
          0.06696489,  0.05739159],
        [-0.00232715, -0.08407147, -0.12328175, ...,  0.00945917,
          0.03173999,  0.11961782]], dtype=float32)]